In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/transactions_scraped.csv',
                 header = None)
df.columns = ["Politician Name", "Party", "Chamber", "State", "Issuer Name", "Ticker", "Publication Date",
              "Transaction Date", "Reporting Gap", "Owner", "Transaction Type", "Value Range", "Price",
              "Transaction ID"]

In [11]:
df.head()

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,Transaction Type,Value Range,Price,Transaction ID,US Ticker,Price_0_days
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO:US,2024-05-01,2024-04-11,19,Spouse,SELL,1K–15K,177.04,10000062280,VLO,177.039993
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB:US,2024-05-01,2024-04-11,19,Spouse,BUY,1K–15K,34.55,10000062281,ENB,34.549999
2,Tom Carper,Democrat,Senate,DE,JPMorgan Chase & Co,JPM:US,2024-05-01,2024-04-01,29,Spouse,SELL,15K–50K,NaN,10000062282,JPM,198.940002
3,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN:US,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,324.40,20003772725,ACN,324.399994
4,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI:US,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,196.33,20003772726,ADI,196.330002


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39114 entries, 0 to 39113
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Politician Name   39114 non-null  object
 1   Party             39114 non-null  object
 2   Chamber           39114 non-null  object
 3   State             39114 non-null  object
 4   Issuer Name       39114 non-null  object
 5   Ticker            35041 non-null  object
 6   Publication Date  39114 non-null  object
 7   Transaction Date  39114 non-null  object
 8   Reporting Gap     39114 non-null  int64 
 9   Owner             39114 non-null  object
 10  Transaction Type  39114 non-null  object
 11  Value Range       39114 non-null  object
 12  Price             33186 non-null  object
 13  Transaction ID    39114 non-null  int64 
dtypes: int64(2), object(12)
memory usage: 4.2+ MB


In [4]:
df[["Politician Name", "Party", "Chamber", "State", "Issuer Name", "Ticker"]] = df[["Politician Name", "Party", "Chamber", "State", "Issuer Name", "Ticker"]].astype('string')

df['Transaction Date'] = df['Transaction Date'].str.replace('Sept', 'Sep')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date']).dt.date

df['Publication Date'] = df['Publication Date'].str.replace('Sept', 'Sep')
df['Publication Date'] = df['Transaction Date'] + pd.to_timedelta(df['Reporting Gap'] + 1, unit='d')

df.Price = df.Price.str.replace(',', '').astype(float)

df['US Ticker'] = df[df['Ticker'].str.contains(':US')].Ticker.str.removesuffix(':US')

<ipython-input-4-f8d2a226c2ee>:7: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['Publication Date'] = df['Transaction Date'] + pd.to_timedelta(df['Reporting Gap'] + 1, unit='d')


In [6]:
df.isna().sum()

Politician Name        0
Party                  0
Chamber                0
State                  0
Issuer Name            0
Ticker                 0
Publication Date       0
Transaction Date       0
Reporting Gap          0
Owner                  0
Transaction Type       0
Value Range            0
Price               1895
Transaction ID         0
dtype: int64

In [12]:
import yfinance as yf
from tqdm import tqdm

def fetch_closest_price(ticker_symbol, date):
    start_date = date - pd.Timedelta(days=3)
    end_date = date + pd.Timedelta(days=3)

    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    period = '1d'

    try:
        data = yf.download(ticker_symbol, start=start_date_str, end=end_date_str, interval=period, progress=False)
        data.dropna(subset=['Close'], inplace=True)
        data['date_diff'] = abs((data.index - date).days)
        data.sort_values('date_diff', inplace=True)

        return data.iloc[0]['Close'] if not data.empty else None
    except Exception as e:
        return None

days_offsets = [0]

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing rows'):
    ticker = row['US Ticker']

    transaction_date = pd.to_datetime(row['Transaction Date'])

    for offset in days_offsets:
        offset_date = transaction_date + pd.Timedelta(days=offset)
        closest_price = fetch_closest_price(ticker, offset_date)

        if closest_price is not None:
            df.at[index, f'Price_{offset}_days'] = closest_price

print("Price updates complete.")

Streaming output truncated to the last 5000 lines.
ERROR:yfinance:['JTSXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['JTSXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['JTSXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
Processing rows:   0%|          | 146/35041 [00:17<27:35, 21.08it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['JTSXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
Processing rows:   0%|          | 150/35041 [00:17<24:25, 23.81it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['JTSXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['OGVXX']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:
1 Failed d

Price updates complete.


In [13]:
df.isna().sum()

Politician Name        0
Party                  0
Chamber                0
State                  0
Issuer Name            0
Ticker                 0
Publication Date       0
Transaction Date       0
Reporting Gap          0
Owner                  0
Transaction Type       0
Value Range            0
Price               1895
Transaction ID         0
US Ticker            106
Price_0_days        1825
dtype: int64

In [14]:
import numpy as np

bins = {
    "< 1K": (0, 1000),
    "1K–15K": (1000, 15000),
    "15K–50K": (15000, 50000),
    "50K–100K": (50000, 100000),
    "100K–250K": (100000, 250000),
    "250K–500K": (250000, 500000),
    "500K–1M": (500000, 1000000),
    "1M–5M": (1000000, 5000000),
    "5M–25M": (5000000, 25000000),
}

midpoints = {bin: (range[0] + range[1]) / 2 for bin, range in bins.items()}

df['Approx Transaction Amount'] = df['Value Range'].map(midpoints)
df['Approx Share Count'] = np.floor(df['Approx Transaction Amount'] / df['Price'])

In [16]:
df.drop(df[df['US Ticker'].isna()].index, inplace=True)
df['Ticker'] = df['US Ticker']
df.drop(['US Ticker'], axis=1, inplace = True)

df.drop(df[df['Price'].isna()].index, inplace=True)
df.drop(df[df['Price'] == 1].index, inplace=True)

df.drop(df[df['Price_0_days'].isna()].index, inplace=True)

In [19]:
df.isna().sum()

Politician Name              0
Party                        0
Chamber                      0
State                        0
Issuer Name                  0
Ticker                       0
Publication Date             0
Transaction Date             0
Reporting Gap                0
Owner                        0
Transaction Type             0
Value Range                  0
Price                        0
Transaction ID               0
Price_0_days                 0
Approx Transaction Amount    0
Approx Share Count           0
dtype: int64

In [20]:
df.head()

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,Transaction Type,Value Range,Price,Transaction ID,Price_0_days,Approx Transaction Amount,Approx Share Count
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO,2024-05-01,2024-04-11,19,Spouse,SELL,1K–15K,177.04,10000062280,177.039993,8000.0,45.0
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB,2024-05-01,2024-04-11,19,Spouse,BUY,1K–15K,34.55,10000062281,34.549999,8000.0,231.0
3,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,324.40,20003772725,324.399994,8000.0,24.0
4,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,196.33,20003772726,196.330002,8000.0,40.0
6,Dan Newhouse,Republican,House,WA,Booking Holdings Inc,BKNG,2024-04-26,2024-04-10,15,Spouse,SELL,1K–15K,3586.13,20003772728,3586.129883,8000.0,2.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31589 entries, 0 to 39112
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Politician Name            31589 non-null  string 
 1   Party                      31589 non-null  string 
 2   Chamber                    31589 non-null  string 
 3   State                      31589 non-null  string 
 4   Issuer Name                31589 non-null  string 
 5   Ticker                     31589 non-null  string 
 6   Publication Date           31589 non-null  object 
 7   Transaction Date           31589 non-null  object 
 8   Reporting Gap              31589 non-null  int64  
 9   Owner                      31589 non-null  object 
 10  Transaction Type           31589 non-null  object 
 11  Value Range                31589 non-null  object 
 12  Price                      31589 non-null  float64
 13  Transaction ID             31589 non-null  int64  


In [23]:
file_path = '/content/drive/MyDrive/Politician Trades/transactions_cleaned.csv'
df.to_csv(file_path, index=False)